In [ ]:
import pyodbc
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

server = os.getenv("SERVER")
database = os.getenv("DATABASE")
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

# Define the connection string for SQL Server (Azure)
conn_str = (
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password}"
)

# Establish the connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connected to the Azure SQL Database successfully!")

except pyodbc.Error as e:
    print(f"Error connecting to the database: {e}")


query = "SELECT * FROM SalesLT.customer"

try:
    # Read data into a DataFrame
    df = pd.read_sql(query, conn)
    print("Data retrieved successfully!")
except Exception as e:
    print(f"Error reading data: {e}")

conn.close()

print(df.head())  # Display the first few rows of the DataFrame

print(df.info())  # Display DataFrame information

Connected to the Azure SQL Database successfully!


C:\Users\angel\AppData\Local\Temp\ipykernel_8984\4098523625.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data retrieved successfully!
   CustomerID  NameStyle Title FirstName MiddleName    LastName Suffix  \
0           1      False   Mr.   Orlando         N.         Gee   None   
1           2      False   Mr.     Keith       None      Harris   None   
2           3      False   Ms.     Donna         F.    Carreras   None   
3           4      False   Ms.     Janet         M.       Gates   None   
4           5      False   Mr.      Lucy       None  Harrington   None   

                  CompanyName               SalesPerson  \
0                A Bike Store   adventure-works\pamela0   
1          Progressive Sports    adventure-works\david8   
2    Advanced Bike Components  adventure-works\jillian0   
3       Modular Cycle Systems  adventure-works\jillian0   
4  Metropolitan Sports Supply      adventure-works\shu0   

                   EmailAddress         Phone  \
0  orlando0@adventure-works.com  245-555-0173   
1    keith0@adventure-works.com  170-555-0127   
2    donna0@adventure-wo

In [12]:
def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df[["FirstName", "ModifiedDate"]].sort_values(by="ModifiedDate")
    df["FirstNameLen"] = df["FirstName"].str.len()
    df["ModifiedDate"] = (
        pd.to_datetime(df["ModifiedDate"]) - pd.to_datetime(df["ModifiedDate"]).min()
    )
    df["ModifiedDate"] = df["ModifiedDate"].dt.days

    return df[["FirstNameLen", "ModifiedDate"]].reset_index(drop=True)

In [6]:
df.head()

,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
0,1,False,Mr.,Orlando,N.,Gee,None,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2005-08-01
1,2,False,Mr.,Keith,None,Harris,None,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2006-08-01
2,3,False,Ms.,Donna,F.,Carreras,None,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2005-09-01
3,4,False,Ms.,Janet,M.,Gates,None,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2006-07-01
4,5,False,Mr.,Lucy,None,Harrington,None,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2006-09-01


In [14]:
# Sort the dataframe by ModifiedDate
df = df.sort_values(by="ModifiedDate")
df.head()

,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
144,227,False,Ms.,Robin,M.,McGuigan,None,"Health Spa, Limited",adventure-works\josé1,robin0@adventure-works.com,431-555-0153,fLh6z9dFddA6FLXlQVHhI5C9QbgD7gDHv63wFwwLzx8=,OtAJEtc=,1FFA0236-84EE-415A-BE61-94CE863715EA,2005-07-01
815,30067,False,Ms.,Phyllis,A.,Thomas,None,Red Bicycle Company,adventure-works\josé1,phyllis2@adventure-works.com,667-555-0112,hAXpzsxtYdB4R3XMc08m98BwHHvL0+J6J+y0u7B1ngg=,ycYA+jA=,67FCFC2C-8E14-45E7-B72D-2D4D7558C454,2005-07-01
407,646,False,Mr.,Scott,None,MacDonald,None,Yellow Bicycle Company,adventure-works\jillian0,scott7@adventure-works.com,470-555-0171,E1wwrSH3sVJUF8UlhuAF+A5UTcdr1LObJBI67CvopDA=,LqtoWXs=,78D632CB-9993-46AC-AA5C-45B2F807F681,2005-07-01
599,29747,False,Ms.,Carolyn,None,Farino,None,The Bike Shop,adventure-works\david8,carolyn0@adventure-works.com,957-555-0125,jnz8GF/glxwiqqdUVHkoV+53Mp1HOnFR/azWzJ80Ktc=,qwoGYjU=,6DA910EF-E6F8-438D-AB3C-82725C41BC6D,2005-07-01
90,146,False,Mr.,Richard,None,Bready,None,Latest Sports Equipment,adventure-works\david8,richard1@adventure-works.com,340-555-0131,kOabrc0OqOISR4N3D3FZkOh3yAKQCF/ozSx7G00Kpt0=,jpRF8lY=,5475E9DD-98CA-4989-B7A2-3FC929BEEA12,2005-07-01


In [13]:
df2 = prepare_data(df)
print(df2.head(100))  # Display the first few rows of the DataFrame

    FirstNameLen  ModifiedDate
0              5             0
1              7             0
2              5             0
3              7             0
4              7             0
..           ...           ...
95             5            31
96             6            31
97             5            31
98             5            31
99             5            31

[100 rows x 2 columns]


In [15]:
# Assign dates to the ModifiedDate column
df3 = df2.copy()
df3["ModifiedDate"] = pd.to_datetime(df2["ModifiedDate"], unit="D", origin="2005-07-01")
df3.head(100)

,FirstNameLen,ModifiedDate
0,5,2005-07-01
1,7,2005-07-01
2,5,2005-07-01
3,7,2005-07-01
4,7,2005-07-01
...,...,...
95,5,2005-08-01
96,6,2005-08-01
97,5,2005-08-01
98,5,2005-08-01


In [16]:
df3.tail()

,FirstNameLen,ModifiedDate
842,7,2009-05-16
843,8,2009-05-16
844,8,2009-05-16
845,3,2009-05-16
846,3,2009-05-16


In [20]:
df.tail(50)

,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
431,689,False,Ms.,Sandra,I.,Martinez,None,Consumer Equipment,adventure-works\jae0,sandra3@adventure-works.com,1 (11) 500 555-0113,zTIlJwPJ9VjXFq2kSSD/H313VF0ZlgeS3a6z5UDvo9k=,Em3q8s4=,AF583E23-093C-4232-B886-A1EE64BCD3C9,2007-09-01 00:00:00.000
548,29660,False,Mr.,Anthony,None,Chor,None,Extreme Riding Supplies,adventure-works\linda3,anthony0@adventure-works.com,429-555-0145,8C7859RNaDWPJkDaaxgaJaZPSf7ikFyFa//RYOz0aPo=,tAAs8Fk=,E24193F1-F5DF-481E-AC1A-BAB86BD72C54,2007-09-01 00:00:00.000
290,454,False,Ms.,Gail,None,Erickson,None,Sleek Bikes,adventure-works\jillian0,gail1@adventure-works.com,834-555-0132,yV4p9H28cp19QO/bcSjLxZFYAs22W5SR9HnmTXUEWkM=,qUowBt4=,D0CC43E1-CF3E-47EA-87F7-2B64C899C56D,2007-09-01 00:00:00.000
423,672,False,Mr.,Gerald,M.,Drury,None,Utilitarian Sporting Goods,adventure-works\shu0,gerald0@adventure-works.com,169-555-0178,Jlxkjapdo78bvTdzddS8yoUxcj6pyFuXwdsCGn6KHvI=,ClRPyyM=,6C3F1514-0E59-467C-AD84-1FCAB936E521,2007-09-01 00:00:00.000
286,448,False,Mr.,Terry,None,Eminhizer,None,Action Bicycle Specialists,adventure-works\jae0,terry1@adventure-works.com,1 (11) 500 555-0176,y8RaWYkfTrcpFEipLXAyKC2pL8AJgQhpW2mOmZ8sqF4=,vx5Ko/Q=,722AEE8F-6B89-4712-BB35-6861AF3065CD,2007-09-01 00:00:00.000
732,29939,False,Mr.,Mark,None,Lee,None,Racing Partners,adventure-works\josé1,mark5@adventure-works.com,371-555-0112,Lk6x4lbbepoCdlNUxLYjn6D3zCWVxEKiK2DwxCURAis=,s7z2FRc=,5BA91047-0E9F-40F1-8D9C-2A493FC9343C,2007-09-01 00:00:00.000
735,29943,False,None,A.,Francesca,Leonetti,None,Two-Seater Bikes,adventure-works\jillian0,a0@adventure-works.com,645-555-0193,M2iP88O+gIF88E9NpBhI0baOUusaGXrum+clJ/miHO8=,6UypM3o=,49E6F552-FB89-4F3B-AE4D-37D9493415D3,2007-09-01 00:00:00.000
168,268,False,Mr.,Richard,A.,Byham,None,Channel Outlet,adventure-works\jae0,richard2@adventure-works.com,1 (11) 500 555-0138,adQ3TmZB28FhQvF2FntW1RInx5S+aOSvwpURQDLarr8=,KoBtEoU=,0C316F6F-5D54-41FD-86A3-6ECAF31A01DD,2007-09-01 00:00:00.000
738,29948,False,Ms.,Elsie,L.,Lewin,None,Town Industries,adventure-works\jillian0,elsie0@adventure-works.com,803-555-0116,sbrtAXJY79C5nTfNaktfR5zn9+uSLMSL26NETFSeEas=,UUwXzgY=,4BF3D7ED-265D-4058-B516-134083182BDB,2007-09-01 00:00:00.000
725,29931,False,Mr.,Eric,None,Lang,None,Kickstands and Accessories Company,adventure-works\pamela0,eric6@adventure-works.com,932-555-0163,katp5sn21ZqJq5Z26YRBk9XXP3iFjpsX7tNPYAAxZlI=,xw9RymE=,7A417F32-0878-45FF-A994-34ED19C5BB11,2007-09-01 00:00:00.000


In [29]:
listan = df2["ModifiedDate"].head().tolist()
listan

[0, 0, 0, 0, 0]

In [30]:
from datetime import date, timedelta


def int_to_date(day_int: int, start_date: date = date(2005, 7, 1)) -> date:
    result = start_date + timedelta(days=day_int)
    result = result.strftime("%Y-%m-%d")
    return result

In [31]:
for i in range(len(listan)):
    listan[i] = int_to_date(listan[i])
listan

['2005-07-01', '2005-07-01', '2005-07-01', '2005-07-01', '2005-07-01']